In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import math
from tqdm import tqdm
import seaborn as sns
import re
import os
import json
import torch
#import isodate
from sklearn.metrics import mean_absolute_error, mean_squared_error
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from surprise import Reader, Dataset, accuracy

C:\Users\lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load CSVs
recipes = pd.read_csv("recipes.csv", dtype={'CookTime': str, 'PrepTime': str, 'TotalTime': str, 'RecipeId': int})
reviews = pd.read_csv("reviews.csv")
print(recipes.shape, reviews.shape)
recipes.head()

(522517, 28) (1401982, 8)


,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [3]:
print(recipes.shape, reviews.shape)
reviews.head()

(522517, 28) (1401982, 8)


,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z


In [4]:
# Drop duplicate recipes/reviews
recipes = recipes.drop_duplicates(subset="RecipeId")
reviews = reviews.drop_duplicates(subset="ReviewId")

nutritional_cols = [
    "Calories","FatContent","SaturatedFatContent","CholesterolContent",
    "SodiumContent","CarbohydrateContent","FiberContent","SugarContent","ProteinContent"
]
for col in nutritional_cols:
    recipes[col] = pd.to_numeric(recipes[col], errors="coerce")

In [5]:
# Negative points look-up tables
energy_points = [80,160,240,320,400,480,560,640,720,800]
sugar_points = [4.5,9,13.5,18,22.5,27,31,36,40,45]
sat_fat_points = [1,2,3,4,5,6,7,8,9,10]
salt_points = [90/400, 180/400, 270/400, 360/400, 450/400, 540/400, 630/400, 720/400, 810/400, 900/400]  

# Positive points tables
fiber_points = [0.7, 1.4, 2.1, 2.8, 3.5]
protein_points = [1.6, 3.2, 4.8, 6.4, 8.0]


def score_from_thresholds(value, thresholds):
    pts = 0
    for t in thresholds:
        if value > t:
            pts += 1
    return pts


def compute_nutriscore(row):
    # NEGATIVE POINTS 
    A = score_from_thresholds(row["Calories"], energy_points)
    B = score_from_thresholds(row["SugarContent"], sugar_points)
    C = score_from_thresholds(row["SaturatedFatContent"], sat_fat_points)

    salt = row["SodiumContent"] / 400  
    D = score_from_thresholds(salt, salt_points)

    N = A + B + C + D

    # POSITIVE POINTS
    # No fruit/veg/nut % available, assume median
    E = 1

    F = score_from_thresholds(row["FiberContent"], fiber_points)
    G = score_from_thresholds(row["ProteinContent"], protein_points)

    if N >= 11:
        G = 0

    P = E + F + G

    total = N - P
    if total <= -1:
        return "A"
    elif total <= 2:
        return "B"
    elif total <= 10:
        return "C"
    elif total <= 18:
        return "D"
    else:
        return "E"


recipes["NutriScore"] = recipes.apply(compute_nutriscore, axis=1)
recipes["NutriScore"].head(20)

0     B
1     E
2     D
3     E
4     C
5     E
6     E
7     C
8     A
9     B
10    E
11    E
12    A
13    A
14    B
15    D
16    E
17    C
18    E
19    A
Name: NutriScore, dtype: object

In [6]:
def precision_recall_at_k(top_n, test_user_items, k=10):
    precisions = []
    recalls = []
    for uid, recs in top_n.items():
        if uid not in test_user_items:
            continue
        relevant = set(test_user_items[uid])
        recommended = set([iid for iid, _ in recs[:k]])
        precisions.append(len(recommended & relevant) / k)
        recalls.append(len(recommended & relevant) / len(relevant))
    return np.mean(precisions), np.mean(recalls)

def ndcg_at_k(top_n, test_user_items, k=10):
    ndcgs = []
    for uid, recs in top_n.items():
        if uid not in test_user_items:
            continue
        relevant = set(test_user_items[uid])
        dcg = 0
        for i, (iid, _) in enumerate(recs[:k]):
            if iid in relevant:
                dcg += 1 / math.log2(i + 2) 
        idcg = sum(1 / math.log2(i + 2) for i in range(min(k, len(relevant))))
        ndcgs.append(dcg / idcg if idcg > 0 else 0)
    return np.mean(ndcgs)

In [7]:
model_name = "microsoft/phi-2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    offload_folder="offload",  
    max_memory={
        0: "6GiB",    # GPU! for an RTX 4050
        "cpu": "16GiB"
    }
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.62s/it]


In [11]:
for name, _ in model.named_modules():
    if "attn" in name or "proj" in name:
        print(name)

model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.dense
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.dense
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.dense
model.layers.3.self_attn
model.layers.3.self_attn.q_proj
model.layers.3.self_attn.k_proj
model.layers.3.self_attn.v_proj
model.layers.3.self_attn.dense
model.layers.4.self_attn
model.layers.4.self_attn.q_proj
model.layers.4.self_attn.k_proj
model.layers.4.self_attn.v_proj
model.layers.4.self_attn.dense
model.layers.5.self_attn
model.layers.5.self_attn.q_proj
model.layers.5.self_attn.k_proj
model.layers.5.self_attn.v_proj
model.layers.5.self_attn.dense
model.layers.6.self_attn
model.layers.6.self_attn.q_proj
model.layers.6.self_attn.k_proj

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "dense"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 5,242,880 || all params: 2,784,926,720 || trainable%: 0.1883


In [ ]:
recipe_names = dict(zip(recipes["RecipeId"], recipes["Name"]))

# Build user histories 
user_histories = defaultdict(list)
for _, row in reviews.iterrows():
    rid = row["RecipeId"]
    if rid in recipe_names:
        user_histories[row["AuthorId"]].append((recipe_names[rid], row["Rating"]))

user_histories = dict(user_histories)

# Save (JSON format)
with open("user_histories.json", "w", encoding="utf-8") as f:
    json.dump(user_histories, f, ensure_ascii=False, indent=2)


In [ ]:
# Load user histories if already saved from before
with open("user_histories.json", "r", encoding="utf-8") as f:
    user_histories = json.load(f)

In [ ]:
samples = []
for _, row in reviews.sample(10000, random_state=None).iterrows():  
    uid, rid, rating = row["AuthorId"], row["RecipeId"], row["Rating"]
    recipe = recipes.loc[recipes["RecipeId"] == rid]
    if recipe.empty:
        continue

    name = recipe.iloc[0]["Name"]
    nutri = recipe.iloc[0]["NutriScore"]
    history = user_histories[uid][-5:] if uid in user_histories else []
    history_text = "\n".join([f"- {rname} ({rscore}/5)" for rname, rscore in history]) or "No previous ratings."

    prompt = (
        "A user has rated several recipes before. Based on their preferences "
        "and the healthiness of the recipe (NutriScore A–E), predict their rating (1–5).\n\n"
        f"User history:\n{history_text}\n\n"
        f"Target recipe: {name} (NutriScore {nutri})\n"
        "Answer only with a single number between 1 and 5 inclusive."
    )

    response = str(int(round(rating)))
    samples.append({"prompt": prompt, "response": response})

In [ ]:
with open("train.jsonl", "w", encoding="utf-8") as f:
    for s in samples[:4000]:
        f.write(json.dumps(s) + "\n")

with open("val.jsonl", "w", encoding="utf-8") as f:
    for s in samples[4000:]:
        f.write(json.dumps(s) + "\n")

In [ ]:
dataset = load_dataset("json", data_files={
    "train": "train.jsonl",
    "validation": "val.jsonl"
})

Generating train split: 4000 examples [00:00, 176598.56 examples/s]
Generating validation split: 5999 examples [00:00, 503363.47 examples/s]


In [ ]:
dataset = load_dataset("json", data_files={"train": "train.jsonl", "validation": "val.jsonl"})

def format_dataset(example):
    text = f"{example['prompt']}\n\n{example['response']}"
    tokenized = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

train_dataset = dataset["train"].map(format_dataset)
val_dataset = dataset["validation"].map(format_dataset)


Map: 100%|██████████| 5999/5999 [00:02<00:00, 2820.65 examples/s]


In [ ]:
training_args = TrainingArguments(
    output_dir="./phi2-finetuned",
    per_device_train_batch_size=1,     
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    eval_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

Step,Training Loss
10,6.545800
20,3.486200
30,0.683100
40,0.441900
50,0.315500
60,0.213800
70,0.130900
80,0.100300
90,0.079300
100,0.061100


TrainOutput(global_step=1000, training_loss=0.1720947505235672, metrics={'train_runtime': 7323.9115, 'train_samples_per_second': 0.546, 'train_steps_per_second': 0.137, 'total_flos': 3.261056679936e+16, 'train_loss': 0.1720947505235672, 'epoch': 1.0})

In [ ]:
model.save_pretrained("./phi2-finetuned")
tokenizer.save_pretrained("./phi2-finetuned")

In [ ]:
base_model_name = "microsoft/phi-2"
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
model = PeftModel.from_pretrained(base_model, "./phi2-finetuned")
model = model.merge_and_unload()

model.save_pretrained("./phi2-merged")
tokenizer.save_pretrained("./phi2-merged")


Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.71s/it]


('./phi2-merged\\tokenizer_config.json',
 './phi2-merged\\special_tokens_map.json',
 './phi2-merged\\vocab.json',
 './phi2-merged\\merges.txt',
 './phi2-merged\\added_tokens.json',
 './phi2-merged\\tokenizer.json')

In [8]:
class LLMPredictor:
    def __init__(self, recipes, user_histories, batch_size=10, model_path="./phi2-merged"):
        self.recipes = recipes
        self.user_histories = user_histories
        self.batch_size = batch_size

        print(f"Loading model from {model_path}...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)

        self.model = AutoModelForCausalLM.from_pretrained(
            model_path,
            dtype=torch.float16,
            device_map="auto"
        )
        self.model.eval()
        print("Model loaded and ready.")

        # Precompute token IDs for digits 1–5
        self.rating_token_ids = {
            i: self.tokenizer.encode(str(i), add_special_tokens=False)[0]
            for i in range(1, 6)
        }

    def _predict_single(self, prompt: str) -> int:
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        )

        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}

        with torch.no_grad():
            output = self.model(**inputs)
            logits = output.logits[:, -1, :]  

        probs = torch.softmax(logits, dim=-1)

        # Choose the rating with the highest probability
        best_rating = max(
            self.rating_token_ids.keys(),
            key=lambda r: probs[0, self.rating_token_ids[r]].item()
        )

        return int(best_rating)

    # Batch prediction for speedup
    def predict_batch(self, pairs):
        preds = []
        for (uid, iid, true_r) in pairs:

            # Get recipe 
            row = self.recipes.loc[self.recipes["RecipeId"] == iid]
            if row.empty:
                preds.append(3)  
                continue

            recipe = row.iloc[0]

            #  User history 
            history_list = self.user_histories.get(uid, [])
            if history_list:
                history_text = "\n".join(
                    f"- {name}: rated {rating}"
                    for name, rating in history_list[-5:]
                )
            else:
                history_text = "No past ratings available."

            #  Build prompt 
            prompt = (
                "You are a model that predicts how much a user will rate a recipe.\n"
                "Respond ONLY with a single number from 1 to 5 (no words).\n"
                "Higher numbers mean the user will like the recipe more.\n"
                "You should also favor recipes with a healthier NutriScore.\n\n"
                f"User ID: {uid}\n\n"
                f"User History:\n{history_text}\n\n"
                f"Recipe:\n"
                f"- Name: {recipe['Name']}\n"
                f"- Health Score (NutriScore): {recipe['NutriScore']}\n\n"
                "Final answer (1–5):"
            )

            # Predict fast
            try:
                pred = self._predict_single(prompt)
            except Exception:
                pred = 3  

            preds.append(pred)

        return preds

In [9]:
reader = Reader(rating_scale=(1, 5))
data = reviews[["AuthorId", "RecipeId", "Rating"]]
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

recipes["RecipeId"] = recipes["RecipeId"].astype(int)
train_df["RecipeId"] = train_df["RecipeId"].astype(int)
test_df["RecipeId"] = test_df["RecipeId"].astype(int)

# Build user histories
user_histories = {}
grouped = train_df.groupby("AuthorId")
for uid, df in tqdm(grouped):
    # Keep recipe name + rating
    user_histories[uid] = [
        (recipes.loc[recipes["RecipeId"] == rid, "Name"].values[0], int(r))
        for rid, r in zip(df["RecipeId"], df["Rating"])
        if rid in recipes["RecipeId"].values
    ]

100%|██████████| 230674/230674 [05:07<00:00, 750.81it/s] 


In [10]:
recipes["RecipeId"] = recipes["RecipeId"].astype(int)
train_df["RecipeId"] = train_df["RecipeId"].astype(int)
test_df["RecipeId"] = test_df["RecipeId"].astype(int)

# Take a smaller sample of the test set
sample_size = 10000  
sampled_test = test_df.sample(n=sample_size, random_state=4).reset_index(drop=True)

# Init model
llm_model = LLMPredictor(recipes, user_histories, batch_size=10, model_path="./phi2-merged")

predictions = []
rows = sampled_test.to_dict(orient="records")

for i in tqdm(range(0, len(rows), llm_model.batch_size)):
    batch = rows[i:i + llm_model.batch_size]
    pairs = [(r["AuthorId"], r["RecipeId"], r["Rating"]) for r in batch]
    ests = llm_model.predict_batch(pairs, )
    for (uid, iid, true_r), est in zip(pairs, ests):
        #print("Estimate for user", uid, "and recipe", iid, "is", est, "true rating is", true_r)
        predictions.append((uid, iid, true_r, est, None))


Loading model from ./phi2-merged...


Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.94s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


Model loaded and ready.


100%|██████████| 1000/1000 [1:32:25<00:00,  5.55s/it]


In [11]:
recipes["RecipeId"] = recipes["RecipeId"].astype(int)
train_df["RecipeId"] = train_df["RecipeId"].astype(int)
test_df["RecipeId"] = test_df["RecipeId"].astype(int)

# Take a smaller sample of the test set
sample_size = 10000  
sampled_test = test_df.sample(n=sample_size, random_state=42).reset_index(drop=True)

model_name = "microsoft/phi-2"

# Init model
llm_model = LLMPredictor(recipes, user_histories, batch_size=10, model_path=model_name)

predictions_base = []
rows = sampled_test.to_dict(orient="records")

for i in tqdm(range(0, len(rows), llm_model.batch_size)):
    batch = rows[i:i + llm_model.batch_size]
    pairs = [(r["AuthorId"], r["RecipeId"], r["Rating"]) for r in batch]
    ests = llm_model.predict_batch(pairs, )
    for (uid, iid, true_r), est in zip(pairs, ests):
        #print("Estimate for user", uid, "and recipe", iid, "is", est, "true rating is", true_r)
        predictions_base.append((uid, iid, true_r, est, None))


Loading model from microsoft/phi-2...


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.27it/s]
Some parameters are on the meta device because they were offloaded to the cpu and disk.


Model loaded and ready.


100%|██████████| 1000/1000 [00:09<00:00, 102.23it/s]


In [12]:
def compute_mae(predictions):
    """Compute Mean Absolute Error."""
    true = [t for _, _, t, _, _ in predictions]
    pred = [p for _, _, _, p, _ in predictions]
    return mean_absolute_error(true, pred)

def compute_rmse(predictions):
    """Compute Root Mean Squared Error."""
    true = [t for _, _, t, _, _ in predictions]
    pred = [p for _, _, _, p, _ in predictions]
    return np.sqrt(mean_squared_error(true, pred))

def compute_metrics(predictions):
    mae = compute_mae(predictions)
    rmse = compute_rmse(predictions)
    print(f"MAE: {mae:.3f}")
    print(f"RMSE: {rmse:.3f}")
    return mae, rmse

print("Finetuned model:")
mae, rmse = compute_metrics(predictions)
print("Base model:")
mae, rmse = compute_metrics(predictions_base)

Finetuned model:
MAE: 0.582
RMSE: 1.385
Base model:
MAE: 1.808
RMSE: 1.899


In [13]:
def get_top_n(predictions, n=10, min_rating=1, max_rating=5):
    top_n = defaultdict(list)

    for uid, iid, true_r, est, _ in predictions:
        # Ignorar predicciones inválidas o nulas
        if est is None:
            continue
        # Asegurar que el rating esté en rango válido
        est = float(est)
        est = min(max(est, min_rating), max_rating)
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return dict(top_n)

In [14]:
test_user_items = defaultdict(list)
for uid, iid, r in zip(test_df["AuthorId"], test_df["RecipeId"], test_df["Rating"]):
    if r >= 4:  # only consider rating 5 as relevant
        test_user_items[uid].append(iid)

In [15]:
top_n = get_top_n(predictions, n=10)
precision, recall = precision_recall_at_k(top_n, test_user_items, k=10)
ndcg = ndcg_at_k(top_n, test_user_items, k=10)

print(f"Precision@10: {precision:.3f}")
print(f"Recall@10:    {recall:.3f}")
print(f"NDCG@10:      {ndcg:.3f}")

Precision@10: 0.144
Recall@10:    0.417
NDCG@10:      0.559


In [16]:
top_n_base = get_top_n(predictions_base, n=10)
precision_base, recall_base = precision_recall_at_k(top_n, test_user_items, k=10)
ndcg_base = ndcg_at_k(top_n, test_user_items, k=10)

print(f"Precision@10: {precision_base:.3f}")
print(f"Recall@10:    {recall_base:.3f}")
print(f"NDCG@10:      {ndcg_base:.3f}")

Precision@10: 0.144
Recall@10:    0.417
NDCG@10:      0.559


In [17]:
nutri_map = {"A": 5, "B": 4, "C": 3, "D": 2, "E": 1}

def mean_health_at_k(top_n, recipes_df, k=10):
    all_scores = []
    for uid, recs in top_n.items():
        for iid, _ in recs[:k]:
            row = recipes_df.loc[recipes_df["RecipeId"] == iid]
            if row.empty:
                continue
            score_letter = row.iloc[0]["NutriScore"]
            score = nutri_map.get(score_letter, np.nan)
            if not np.isnan(score):
                all_scores.append(score)
    
    return np.mean(all_scores) if all_scores else np.nan

# top_n = get_top_n(predictions, n=10)
mean_health = mean_health_at_k(top_n, recipes, k=10)
print(f"MeanHealth@10: {mean_health:.4f}")
mean_health_base = mean_health_at_k(top_n_base, recipes, k=10)
print(f"MeanHealth@10 (base): {mean_health_base:.4f}")

MeanHealth@10: 2.5635
MeanHealth@10 (base): 2.5664


In [18]:
def show_example_recommendation(uid, predictor, interactions_df, recipes_df, n_candidates=20):

    # Get user history
    history = predictor.user_histories.get(uid, [])

    print(f"RECOMMENDATION FOR USER {uid}")

    if history:
        print("User History (last 5):")
        for name, rating in history[-5:]:
            print(f" - {name}: rated {rating}")
    else:
        print("User has no history.")

    # Select candidate recipes the user has NOT rated
    already_rated = set(interactions_df[interactions_df["AuthorId"] == uid]["RecipeId"])

    candidate_pool = recipes_df[~recipes_df["RecipeId"].isin(already_rated)]

    candidate_recipes = candidate_pool.sample(min(n_candidates, len(candidate_pool)), random_state=42)

    # Predict scores
    pairs = [(uid, iid, None) for iid in candidate_recipes["RecipeId"]]
    preds = predictor.predict_batch(pairs)

    results = list(zip(
        candidate_recipes["RecipeId"].tolist(),
        candidate_recipes["Name"].tolist(),
        preds,
        candidate_recipes["NutriScore"].tolist()
    ))
    
    # Sort by predicted rating descending
    results.sort(key=lambda x: x[2], reverse=True)

    # Output top recommendation
    best_iid, best_name, best_score, health_score = results[0]

    print("\nTop Recommended Recipe:")
    print(f" - Name: {best_name}")
    print(f" - RecipeId: {best_iid}")
    print(f" - Predicted Rating: {best_score}")
    print(f" - Health Score (NutriScore): {health_score}")

    print("\nOther candidates:")
    for rid, name, score in results[1:5]:  # show next 4
        print(f" - {name} (id={rid}),  score {score}")


In [ ]:
uid = reviews["AuthorId"].sample(1).iloc[0]
predictor = LLMPredictor(recipes, user_histories, batch_size=1, model_path="./phi2-merged")
show_example_recommendation(uid, predictor, reviews, recipes)

Loading model from ./phi2-merged...


Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.49s/it]
Some parameters are on the meta device because they were offloaded to the cpu and disk.


Model loaded and ready.
RECOMMENDATION FOR USER 875605
User History (last 5):
 - Very Simple Blueberry Muffins: rated 5
 - Vegan Cornbread!: rated 5
